In [2]:
from fetch_news import *
import yaml
import io

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import os.path
from datetime import datetime, timedelta
import datetime
import aiohttp
import asyncio

In [3]:
# from datetime import datetime
import datetime

from sqlalchemy.orm import Session
from sqlalchemy.sql import func
import models
import schemas
import crud
from database import SessionLocal, engine, SQLALCHEMY_DATABASE_URL

import psycopg2

import json
import os

In [3]:
import importlib
importlib.reload(crud)

In [5]:
API_FILE = "api_tracker.yaml"
with open(API_FILE) as f:
    generator = yaml.safe_load(f)
    APIS = list(generator)
    APIS = list(map(lambda x: tuple(x.items())[0], APIS))

In [6]:
for api in APIS[:2]:
    print(api)

('API Key 1', {'owner': 'Kleon', 'start_index': 0, 'end_index': 99, 'key': 'uvi3iw3bL4jaROoaT0TGgbu0HRr9sQc9mWyMhFCa'})
('API Key 2', {'owner': 'Kleon', 'start_index': 100, 'end_index': 199, 'key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7'})


In [7]:
len(APIS)

23

In [26]:
start_idx = 101
end_idx = 102

In [27]:
# db = SessionLocal()

In [28]:
# def get_all_stock_tickers(db: Session):
#     return db.query(models.Stock, func.row)

In [29]:
# sql_query = f'''
# SELECT *
# FROM (SELECT ROW_NUMBER() OVER (ORDER BY ticker) as idx, ticker 
# 	FROM stocks) as t
# WHERE idx BETWEEN {start_idx} AND {end_idx};
# '''

In [30]:
# result = db.execute(sql_query).all()

In [31]:
# result

In [32]:
def get_request_param(APIS, stocks):
    request_param = []
    for idx, ticker in stocks:
        is_found = False
        param = {
            "idx": idx,
            "ticker": ticker
        }
        for api_key, api_param in APIS:
            # print(idx, api_param["start_index"],  api_param["end_index"])
            if api_param["start_index"] <= idx <= api_param["end_index"]:
                param["api_key_name"] = api_key
                param["api_key"] = api_param["key"]
                request_param.append(param)
                is_found = True
                break;
            elif api_param["start_index"] == "*" and api_param["end_index"] == "*":
                param["api_key_name"] = api_key
                param["api_key"] = api_param["key"]
                request_param.append(param)
                is_found = True
                break;
            elif api_param["start_index"] <= idx and api_param["end_index"] == "*":
                param["api_key_name"] = api_key
                param["api_key"] = api_param["key"]
                request_param.append(param)
                is_found = True
                break;
            elif api_param["start_index"] == "*" and idx <= api_param["end_index"] :
                param["api_key_name"] = api_key
                param["api_key"] = api_param["key"]
                request_param.append(param)
                is_found = True
                break;
        
        if not is_found:
            raise Exception(f"Failed to allocate API key for index {idx}: {ticker}")
    return request_param


In [33]:
try:
  db = SessionLocal()
  sql_query = f'''
    SELECT *
    FROM (SELECT ROW_NUMBER() OVER (ORDER BY ticker) as idx, ticker 
      FROM stocks) as t
    WHERE idx BETWEEN {start_idx} AND {end_idx};
  '''
  stocks = db.execute(sql_query).all()
  request_param = get_request_param(APIS, stocks)
  print("Length of request param:",len(request_param))
  print(request_param[:2])
finally:
  db.close()

Length of request param: 2
[{'idx': 101, 'ticker': 'AMEH', 'api_key_name': 'API Key 2', 'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7'}, {'idx': 102, 'ticker': 'AMG', 'api_key_name': 'API Key 2', 'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7'}]


In [34]:
try:
  db = SessionLocal()

  for idx in range(len(request_param)):
    param = request_param[idx]
    news = crud.get_last_news(db, param["ticker"])
    if len(news) == 0:
      date = None
      start_date = datetime.datetime.today() - timedelta(weeks=2)
    else:
      date = news["date"]
      # date.strftime("%Y-%m-%d")
      start_date = (date + timedelta(days=1)).replace(hour = 0, minute = 1, second = 0)
    print(param["ticker"], date, start_date)
    param["start_date"] = start_date      
finally:
  db.close()
print("Insert start dates")
print(request_param[:2])

AMEH None 2022-10-28 23:38:00.879041
AMG 2021-01-14 17:18:27 2021-01-15 00:01:00
Insert start dates
[{'idx': 101, 'ticker': 'AMEH', 'api_key_name': 'API Key 2', 'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7', 'start_date': datetime.datetime(2022, 10, 28, 23, 38, 0, 879041)}, {'idx': 102, 'ticker': 'AMG', 'api_key_name': 'API Key 2', 'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7', 'start_date': datetime.datetime(2021, 1, 15, 0, 1)}]


In [19]:
request_param[:2]

[{'idx': 100,
  'ticker': 'AMED',
  'api_key_name': 'API Key 2',
  'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7',
  'start_date': datetime.datetime(2022, 10, 28, 23, 36, 54, 595249)},
 {'idx': 101,
  'ticker': 'AMEH',
  'api_key_name': 'API Key 2',
  'api_key': 'QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7',
  'start_date': datetime.datetime(2022, 10, 28, 23, 36, 54, 685754)}]

In [46]:
s = request_param[0]["start_date"] - datetime.timedelta(weeks=10)
e = min(s + datetime.timedelta(weeks=2), datetime.datetime.today())

In [47]:
print(s)
print(e)

2022-08-19 23:38:00.879041
2022-09-02 23:38:00.879041


# Fetch News

In [48]:
async def fetch_token_news(session, ticker, token, start_date, end_date):
    tasks = []
    start_date_formated = start_date.strftime("%Y-%m-%d") 
    end_date_formated = end_date.strftime("%Y-%m-%d")
    try:
      url = f"https://api.marketaux.com/v1/news/all?symbols={ticker}&api_token={token}&filter_entities=true&published_before={end_date}&published_after={start_date}"
      async with session.get(url) as response:
        result = await response.json()
        temp = {
          "ticker": ticker,
          "token": token,
          "start_date": start_date,
          "end_date": end_date,
          "news": result
        }
        tasks.append(temp)
    except:
      print(f"Fail[{ticker}]: {start_date} - {end_date}")
    return tasks

async def fetch_all_news_data_async(request_param):
  news = {}
  tasks = []
  async with aiohttp.ClientSession() as session:
    counter = 0
    for idx in range(len(request_param)):
      param = request_param[idx]
      ticker = param["ticker"]
      token = param["api_key"]
      start_date = param["start_date"]
      end_date = min(start_date + datetime.timedelta(weeks=2), datetime.datetime.today())
      tasks.append(fetch_token_news(session, ticker, token, start_date, end_date))
      counter += 1
      if counter == 6:
        await asyncio.sleep(300)

    responses = await asyncio.gather(*tasks)
    for response in responses:
      for result in response:
        try:
          ticker = result["data"][0]["entities"][0]["symbol"]
          news[ticker] = result
        except Exception as err:
          print(f"Rate limit reached")
  return news

def process_news_data(news_dict):
  df = pd.DataFrame()
  for result in news_dict:
    news = result["news"]
    ticker = result["ticker"]
    title = list(map(lambda x: x['title'], news['data']))
    pub_date = list(map(lambda x: x['published_at'], news['data']))
    source = list(map(lambda x: x['source'], news['data']))
    url = list(map(lambda x: x['url'], news['data']))
    entities = list(map(lambda x: list(filter(lambda y: y['symbol'] == ticker, x['entities']))[0], news['data']))
    match_score = list(map(lambda x: x["match_score"], entities))
    sentiment_score = list(map(lambda x: x["sentiment_score"], entities))

    def process_text(text):
      text = text.replace('\n\n', '... ')
      text = BeautifulSoup(text, 'html.parser').get_text()
      return text
    text = list(map(lambda y: '\n'.join(list(map(lambda z: process_text(z['highlight']), y))), list(map(lambda x: x['highlights'], entities))))
    
    row = {
        "Stock": [ticker] * len(title), 
        "Title": title, 
        "Date/Time Published": pub_date, 
        "Source": source, 
        "Url": url,
        "Relevant Texts": text,
        "Sentiment Score": sentiment_score,
        "Match Score": match_score
        }
    temp = pd.DataFrame(row)
    df = df.append(temp, ignore_index = True)
  return df

In [58]:
df = pd.DataFrame(stocks, columns=["index", 'ticker']).set_index("index")

In [59]:
df

,ticker
index,
101,AMEH
102,AMG


In [60]:
df.index

Int64Index([101, 102], dtype='int64', name='index')

In [63]:
df.loc[101:102]

,ticker
index,
101,AMEH
102,AMG


In [66]:
df.loc[50:51, "ticker"].to_numpy()

array([], dtype=object)

In [50]:
task = []
idx = 0
with aiohttp.ClientSession() as session:
    param = request_param[idx]
    ticker = param["ticker"]
    token = param["api_key"]
    start_date = param["start_date"]
    end_date = min(start_date + datetime.timedelta(weeks=2), datetime.datetime.today())
    task = fetch_token_news(session, ticker, token, start_date, end_date)

TypeError: Use async with instead

In [49]:
news = asyncio.run(fetch_all_news_data_async(request_param=request_param))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
# db.close()

In [ ]:
stock_ticker = 

# Get the Latest News Date

In [7]:
# Dependency
def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()

In [3]:
db = SessionLocal()

In [4]:
news = crud.get_last_news(db, "AMG")

In [10]:
date = news["date"]

In [13]:
date.strftime("%d/%m/%Y")

'14/01/2021'

In [21]:
start_date = (date + datetime.timedelta(days=1)).replace(hour = 0, minute = 1, second = 0)

In [24]:
start_date

datetime.datetime(2021, 1, 15, 0, 1)

In [23]:
db.close()

# Test 1

In [4]:
start_idx = 101
end_idx = 102
print(start_idx, end_idx)

API_FILE = "api_tracker.yaml"
with open(API_FILE) as f:
    generator = yaml.safe_load(f)
    APIS = list(generator)
    APIS = list(map(lambda x: tuple(x.items())[0], APIS))
# for api in APIS[:2]:
#     print(api)


API_KEYS = [val["key"] for key, val in APIS] # pass in the list of API tokens
start_indexes = [val["start_index"] for key, val in APIS] # pass in the list of start index corresponding to each API token
# API_KEYS = ["uvi3iw3bL4jaROoaT0TGgbu0HRr9sQc9mWyMhFCa", "QAtmHlReb9ZdjaicSgdpARuHGYVYJgzjYuSiFXD7"]
# start_indexes = [0, 100]
no_weeks = 20 # past n weeks of data from today to be collected (need to be a sufficiently distant past)

stocks = []
try:
  db = SessionLocal()
  sql_query = f'''
    SELECT *
    FROM (SELECT ROW_NUMBER() OVER (ORDER BY ticker) as idx, ticker 
      FROM stocks) as t
    WHERE idx BETWEEN {start_idx} AND {end_idx};
  '''
  stocks = db.execute(sql_query).all()
  # request_param = get_request_param(APIS, stocks)
  # print("Length of request param:",len(request_param))
  # print(request_param[:2])
finally:
  db.close()
# print(stocks)
df = pd.DataFrame(stocks, columns=["index", 'ticker']).set_index("index")
# print(df)

# df["start_date"] = np.nan
# print(df.head(5))

# try:
#     db = SessionLocal()

#     for idx in df.index:
#       ticker = df.loc[idx, "ticker"]
#       news = crud.get_last_news(db, ticker)
#       if len(news) == 0:
#         date = None
#         start_date = (datetime.datetime.today() - timedelta(weeks=2)).replace(hour=0, minute=1, second=0)
#       else:
#         date = news["date"]
#         # date.strftime("%Y-%m-%d")
#         start_date = (date + timedelta(days=1)).replace(hour = 0, minute = 1, second = 0)
#       print(ticker, date, start_date)
#       df.loc[idx, "start_date"] = start_date  
# finally:
#   db.close()
# print("Insert start dates")
# print(df.head(5))

start_date = (datetime.datetime.today() - timedelta(weeks=no_weeks)).strftime("%Y-%m-%d")

news = asyncio.run(fetch_all_news_data_async(df, start_date))
news_df = process_news_data(news)

csv_name = f"test_data/marketaux_newsdata_async.csv"
filename = csv_name

101 102


RuntimeError: asyncio.run() cannot be called from a running event loop